In [33]:
import numpy as np
import pandas as pd
#
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
#
import plotly.graph_objects as go


# Read in price data
df = pd.read_csv('../../data/df_monthly_returns_complete.csv', index_col='Date')
#df = df[['TEG.DE', 'CBRE', 'PLXS', 'JLL', 'CDW', 'TPL', 'FN', 'SNX', 'BYG.L', 'AVB', 'RGLD', 'ACN', 'ITRI', 'WDC', 'SBAC', 'LIN', 'SWBI', 'AZO']]
df = df[df.columns[0:200]] # max 200

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(df, frequency=12)
S = risk_models.sample_cov(df, frequency=12)

# Optimize for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
ef_new = EfficientFrontier(mu, S)

raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.csv")  # saves to file
#
ef.portfolio_performance(verbose=True)

Expected annual return: 16.1%
Annual volatility: 11.6%
Sharpe Ratio: 1.22


(0.16089413116773077, 0.11591987511217045, 1.2154441249302061)

In [34]:
latest_prices = get_latest_prices(df)

da = DiscreteAllocation(raw_weights, latest_prices, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: €{:.2f}".format(leftover))

Discrete allocation: {'AZO': 1, 'PHP.L': 12, 'BMI': 3, 'DHR': 2, 'TPL': 1, 'TEP.L': 1, 'POOL': 2, 'BALL': 8, 'RGLD': 3, 'CDW': 1, 'INS.DE': 12}
Funds remaining: €1.89


In [45]:
# Generate efficient frontier data
target_returns = np.linspace(mu.min(), mu.max()-0.001, 200)
frontier_volatility = []
frontier_returns = []
print(len(target_returns))
for r in target_returns:
    ab = ef_new.efficient_return(target_return=r)
    ret, vol, _ = ef_new.portfolio_performance()
    frontier_returns.append(ret)
    frontier_volatility.append(vol)

200


In [46]:
mu.max()

0.26410942626166367

In [47]:
import plotly.graph_objects as go

# Create a Plotly scatter plot
fig = go.Figure()

# Convert expected returns to a NumPy array
returns_array = mu.values

# Calculate volatilities (square root of the diagonal of covariance matrix)
volatilities_array = np.sqrt(np.diag(S))

# Marker: Add annotations for each point
for i, (ret, vol, ticker) in enumerate(zip(returns_array, volatilities_array, mu.keys())):
    fig.add_trace(go.Scatter(
        x=[vol],
        y=[ret],
        mode='markers+text',
        text=f"{ticker} ({ret:.2%}, {vol:.2%})",
        textposition="top center",
        marker=dict(size=6, color='blue'),
        showlegend=False
    ))

max_sharpe_return, max_sharpe_volatility, _ = ef.portfolio_performance()
# Marker: Add the maximum Sharpe ratio portfolio
fig.add_trace(go.Scatter(
    x=[max_sharpe_volatility],
    y=[max_sharpe_return],
    mode='markers',
    name='Max Sharpe Portfolio',
    marker=dict(color='red', size=10, symbol='star')
))

# Line: Add the efficient frontier
fig.add_trace(go.Scatter(
    x=frontier_volatility,
    y=frontier_returns,
    mode='lines',
    name='Efficient Frontier',
    line=dict(color='blue', width=3)
))

#fig_ef.update_traces(hovertemplate=None)
# Customize layout
fig.update_layout(
    title="Efficient Frontier",
    xaxis_title="Volatility (Standard Deviation)",
    yaxis_title="Expected Return",
    template="plotly_white"
)

fig.show()

### Allocation